# Flags

> Flags are custom ipython magics, that tell nbdev how to process cells in a notebook.

In this notebook, we test the flags defined in `flags.py` and show how you can start using them in your notebooks.

## Note: Magic flags are being introduced progressively

In order to explore autocomplete and in-notebook help with magic flags we plan to;
- start by making `%nbdev_export` available as an alternative to `#export`
- support both comments and magic flags


## Using magic flags

You can start using magic flags in existing notebooks by running the `nbdev_upgrade` console command.

If you prefer not to use the `nbdev_upgrade` console command, you'll need to "import" flags before you can use them. Any nbdev import will import all flags via `nbdev.__init__.py`. So you could use
- `import nbdev` or
- `from nbdev.imports import *`

but if you want the convenience of importing `show_doc` and `notebook2script` as well, use

In [ ]:
from nbdev import *

Once imported, you can explore the available flags and their documentation without leaving your notebook

<img alt="Tab completion of nbdev magics" width="900" src="images/inspect_magics.png" />

Note: In the image above, we show some flags that have not yet been introduced to nbdev.

You can also see that some flags print warnings if used incorrectly

<pre>
&percnt;nbdev_export bad module name
</pre>

<pre>UsageError: module_name "bad module name" must not contain whitespace</pre>

## Tests

### We can test magics in ipython ...

In [ ]:
from nbdev.imports import *
from contextlib import redirect_stdout
import io

if IN_IPYTHON:
    def test_flag_output(s, exp): test_eq(s.getvalue().strip(), exp.strip())
    s = io.StringIO()
    with redirect_stdout(s):
        %nbdev_export
        test_flag_output(s, '')
        %nbdev_export other.module
        test_flag_output(s, '')
        %nbdev_export bad module name
        test_flag_output(s, 'UsageError: module_name "bad module name" must not contain whitespace')

### ... but we need less code to test in normal python

Note: We don't `del` the functions in `flags.py` so that we can test them here

In [ ]:
def test_flag_output(f, line, exp):
    "Test that `f(line)` prints `exp` to stdout"
    s = io.StringIO()
    with redirect_stdout(s): f(line)
    test_eq(s.getvalue().strip(), exp.strip())

from nbdev.flags import nbdev_default_export
test_flag_output(nbdev_default_export, '', 'UsageError: module_name is missing. Usage `%nbdev_default_export module_name`')
test_flag_output(nbdev_default_export, '  ', 'UsageError: module_name is missing. Usage `%nbdev_default_export module_name`')
test_flag_output(nbdev_default_export, 'bad module name', 'UsageError: module_name "bad module name" must not contain whitespace')
test_flag_output(nbdev_default_export, 'special.module', 
            'Cells will be exported to nbdev.special.module,\n' +
            'unless a different module is specified after an export flag: `%nbdev_export special.module`')

from nbdev.flags import nbdev_export, nbdev_export_and_show, nbdev_export_internal
for f in [nbdev_export, nbdev_export_and_show, nbdev_export_internal]:
    test_flag_output(f, '', '')
    test_flag_output(f, ' ', '')
    test_flag_output(f, 'special.module', '')
    test_flag_output(f, 'bad module name', 'UsageError: module_name "bad module name" must not contain whitespace')

from nbdev.flags import nbdev_hide, nbdev_hide_input, nbdev_hide_output
for f in [nbdev_hide, nbdev_hide_input, nbdev_hide_output]:
    test_flag_output(f, '', '')
    test_flag_output(f, 'anything', f'UsageError: Invalid option "anything". Usage `%{f.__name__}`')
    
from nbdev.flags import nbdev_default_class_level
for i in range(1,7):
    test_flag_output(nbdev_default_class_level, str(i), '')
test_flag_output(nbdev_default_class_level, '0', 'UsageError: Invalid class level "0". Usage `%nbdev_default_class_level [int between 1 and 6]`')
test_flag_output(nbdev_default_class_level, '7', 'UsageError: Invalid class level "7". Usage `%nbdev_default_class_level [int between 1 and 6]`')
test_flag_output(nbdev_default_class_level, 'not_a_num', 'UsageError: Invalid class level "not_a_num". Usage `%nbdev_default_class_level [int between 1 and 6]`')

from nbdev.flags import nbdev_fastai2_test
test_flag_output(nbdev_fastai2_test, '', '')
test_flag_output(nbdev_fastai2_test, ' ', '')
test_flag_output(nbdev_fastai2_test, 'all', '')
test_flag_output(nbdev_fastai2_test, 'All', 'UsageError: Invalid option "All". Usage `%nbdev_fastai2_test [all]`')

from nbdev.flags import nbdev_collapse_input, nbdev_collapse_output
for f in [nbdev_collapse_input, nbdev_collapse_output]:
    test_flag_output(f, '', '')
    test_flag_output(f, ' open ', '')
    test_flag_output(f, 'open', '')
    test_flag_output(f, 'OPEN', f'UsageError: Invalid option "OPEN". Usage `%{f.__name__} [open]`')

In [ ]:
%nbdev_hide
from nbdev.flags import parse_line
for expected, line in [
        [['name'], 'name'],
        [['name', '"name2"'], 'name "name2"'],
        [['"name"', 'name2'], '"name", name2'],
        [['"name"', 'name2', 'name3'], '"name", name2 name3'],
        [['"name"', 'name2', 'name3'], ' , , "name", , name2 name3,,  '],
        [['"name"', 'name2'], '["name", name2]'],
        [['name', "'name2'"], " [ name, 'name2' ]"]]:
    test_eq(expected, parse_line(line))

In [ ]:
from nbdev.flags import nbdev_add2all
fn = partial(nbdev_add2all, local_ns=globals())
test_flag_output(fn, '', 'UsageError: List of names is missing. Usage `%nbdev_add2all name_1, name_2`')
test_flag_output(fn, 'str int max', '')
test_flag_output(fn, 'str,int,max,', '')
test_flag_output(fn, 'str, int, max', '')
test_flag_output(fn, ' [ str ,,,"int" max ]', '')
test_flag_output(fn, '[str, "int", max ', 'UsageError: unexpected EOF while parsing (<string>, line 1)')
test_flag_output(fn, '"bad_name" max', '')
test_flag_output(fn, 'bad_name max', 'UsageError: name \'bad_name\' is not defined')
test_flag_output(fn, '"bad_name max', 'UsageError: EOL while scanning string literal (<string>, line 1)')

In [ ]:
from nbdev.showdoc import nbdev_show_doc
fn = partial(nbdev_show_doc, local_ns=globals())
test_flag_output(fn, '', 'UsageError: List of names is missing. Usage `%nbdev_show_doc name_1, name_2`')
test_flag_output(fn, 'title_level=1', 'UsageError: List of names is missing. Usage `%nbdev_show_doc name_1, name_2`')
test_flag_output(fn, 'title_level=0 default_cls_level=2', 
"""UsageError: Invalid title_level "0". Usage `%nbdev_show_doc name_1 title_level=[int between 1 and 6]`
UsageError: List of names is missing. Usage `%nbdev_show_doc name_1, name_2`""")
test_flag_output(fn, 'title_level=1 default_cls_level=7', 
"""UsageError: Invalid default_cls_level "7". Usage `%nbdev_show_doc name_1 default_cls_level=[int between 1 and 6]`
UsageError: List of names is missing. Usage `%nbdev_show_doc name_1, name_2`""")

## Export -

In [ ]:
%nbdev_hide
notebook2script()

Converted 00_export.ipynb.
Converted 01_sync.ipynb.
Converted 02_showdoc.ipynb.
Converted 03_export2html.ipynb.
Converted 04_test.ipynb.
Converted 05_merge.ipynb.
Converted 06_cli.ipynb.
Converted 07_clean.ipynb.
Converted 08_flag_tests.ipynb.
Converted 99_search.ipynb.
Converted index.ipynb.
Converted tutorial.ipynb.
